# Met weather

In [2]:
import requests
import geonamescache
from datetime import datetime
import pytz

## Get lat lon to supply to weather url

In [3]:
# Create an instance of the GeoNamesCache
gc = geonamescache.GeonamesCache()

In [4]:
# Retrieve cities data
cities = gc.get_cities()

# Specify the city name you want to search for
city_name = "Denver"

In [5]:
# Search for the city in the cities data
for city_code, city_data in cities.items():
    if city_data['name'] == city_name:
        # Extract latitude and longitude
        latitude = city_data['latitude']
        longitude = city_data['longitude']
        print(f"Latitude: {latitude}, Longitude: {longitude}")
        break
else:
    print(f"Could not find city name: {city_name}")

Latitude: 39.73915, Longitude: -104.9847


## Get weather data

In [6]:
# Specify the MET API URL with the retrieved latitude and longitude
url = f"https://api.met.no/weatherapi/locationforecast/2.0/compact?lat={latitude}&lon={longitude}"

# Define a custom User-Agent header
headers = {
    "User-Agent": "YourAppName/1.0 (your-email@example.com) _ 11298370iahdjhiaosfdi8y98 y9801"
}

In [7]:
# Make an API request to the MET API with the custom User-Agent header
response = requests.get(url, headers=headers)

# Parse the JSON response
weather_data = response.json()

### Helper Functions

In [8]:
# Helper function to convert Celsius to Fahrenheit
def celsius_to_fahrenheit(celsius):
    return celsius * 9 / 5 + 32

# Helper function to convert meters per second to miles per hour
def ms_to_mph(meters_per_second):
    return meters_per_second * 2.237

# Helper function to convert wind direction in degrees to compass direction
def wind_direction_to_compass(degrees):
    compass_points = [
        "N", "NNE", "NE", "ENE", "E", "ESE", "SE", "SSE",
        "S", "SSW", "SW", "WSW", "W", "WNW", "NW", "NNW"
    ]
    # Calculate the index in the compass_points list based on the degrees
    index = int((degrees + 11.25) / 22.5) % 16
    return compass_points[index]



# Function to convert UTC timestamp to Mountain Time (Denver, CO)
def convert_to_mountain_time(utc_timestamp, format="%Y-%m-%d %H:%M"):

    # Define the UTC and Mountain Time (Denver, CO) time zones
    utc_timezone = pytz.utc
    mountain_timezone = pytz.timezone('America/Denver')


    # Parse the UTC timestamp to a datetime object
    utc_time = datetime.fromisoformat(utc_timestamp.rstrip('Z'))
    
    # Make the datetime object timezone aware (UTC timezone)
    utc_time = utc_timezone.localize(utc_time)
    
    # Convert the UTC time to Mountain Time (Denver, CO)
    mountain_time = utc_time.astimezone(mountain_timezone)
    
    # Return the Mountain Time in the specified format
    return mountain_time.strftime(format)

### Parse weather data function

In [9]:
# Function to parse the forecasted weather data
def parse_weather_data(data):
    timeseries = data['properties']['timeseries']
    
    # Iterate through each time series entry
    for entry in timeseries:
        time = convert_to_mountain_time(entry['time'])
        instant_data = entry['data']['instant']['details']
        
        # Current instant data
        current_temp_celsius = instant_data['air_temperature']
        current_temp_fahrenheit = celsius_to_fahrenheit(current_temp_celsius)
        current_wind_speed_ms = instant_data['wind_speed']
        current_wind_speed_mph = ms_to_mph(current_wind_speed_ms)
        current_wind_direction = instant_data['wind_from_direction']
        # Convert wind direction in degrees to compass direction
        compass_direction = wind_direction_to_compass(current_wind_direction)

        # Get symbol code for instant data from the next 1-hour forecast data
        if 'next_1_hours' in entry['data']:
            next_1_hour = entry['data']['next_1_hours']
            symbol_code_instant = next_1_hour['summary']['symbol_code']
        else:
            symbol_code_instant = 'N/A'  # If next 1-hour forecast data is unavailable

        # Retrieve precipitation data for the next 1, 6, and 12 hours
        precip_next_1_hour = next_1_hour['details']['precipitation_amount'] if 'next_1_hours' in entry['data'] and 'details' in entry['data']['next_1_hours'] else 0.0
        precip_next_6_hour = entry['data']['next_6_hours']['details']['precipitation_amount'] if 'next_6_hours' in entry['data'] and 'details' in entry['data']['next_6_hours'] else 0.0
        

        # Print
        prnt_str_time = f"Time: {time} - {symbol_code_instant}\n"
        prnt_str_weather_0 = f"Temperature: {current_temp_fahrenheit:.2f}°F, Wind speed: {current_wind_speed_mph:.2f} mph, Wind direction: {compass_direction}\n"
        prnt_str_weather_1 = f"Precip next 1 hour: {precip_next_1_hour} mm, Precip next 6 hours: {precip_next_6_hour} mm\n"

        print(prnt_str_time + prnt_str_weather_0 + prnt_str_weather_1)

In [10]:
parse_weather_data(weather_data)

Time: 2024-04-16 17:00 - clearsky_day
Temperature: 65.48°F, Wind speed: 14.99 mph, Wind direction: WNW
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 18:00 - clearsky_day
Temperature: 63.50°F, Wind speed: 17.90 mph, Wind direction: WNW
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 19:00 - clearsky_day
Temperature: 60.26°F, Wind speed: 16.55 mph, Wind direction: W
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 20:00 - clearsky_night
Temperature: 55.76°F, Wind speed: 13.87 mph, Wind direction: W
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 21:00 - clearsky_night
Temperature: 51.08°F, Wind speed: 6.26 mph, Wind direction: S
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 22:00 - clearsky_night
Temperature: 48.74°F, Wind speed: 10.51 mph, Wind direction: S
Precip next 1 hour: 0.0 mm, Precip next 6 hours: 0.0 mm

Time: 2024-04-16 23:00 - partlycloudy_night